# core

> Config package for python

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import yaml
import os
import pathlib

In [ ]:
#| exporti
def get_env(
        var:str, # The environment variable to get
        default:str = "default" # The default value to return if the environment variable is not set
    ):
    "Get a value from the environment variables."
    return os.environ.get(var, "default")

assert get_env('TEST') == 'default'
os.environ['TEST'] = 'test'
assert get_env('TEST') == 'test'



In [ ]:
#| export
def config_get(
        value: str,
        py_config_active:str = None,
        file:str = 'config.yaml',
        encoding:str = 'utf-8'
    ):
    "Get a value from the config.yaml file."
    if py_config_active is None:
        py_config_active = get_env('R_CONFIG_ACTIVE', 'default')
    
    dir_path = os.getcwd()
    file = pathlib.Path(dir_path, file)

    with open(file, 'r') as stream:
        conf = yaml.safe_load(stream)
    return conf[py_config_active][value]



In [ ]:
assert config_get('trials') == 5
assert config_get('trials', 'production') == 30

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()